In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk
import math

data = pd.read_csv("dataAfterPOS.csv")
data

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,word_count
0,Italy,aromas tropical fruit broom brimstone herb pal...,Vulkà Bianco,87,17.000000,Sicily & Sardinia,Etna,unknown,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,24
1,Portugal,ripe fruity wine firm tannins juicy red berry ...,Avidagos,87,15.000000,Douro,unknown,unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,39
2,US,tart snappy flavors lime flesh dominate green ...,unknown,87,14.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,28
3,US,pineapple rind lemon pith orange blossom aroma...,Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,unknown,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,33
4,US,much regular bottling rough tannic rustic eart...,Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,41
5,Spain,blackberry typical navarran whiff green herbs ...,Ars In Vitro,87,15.000000,Northern Spain,Navarra,unknown,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,42
6,Italy,bright informal aromas candied berry white pep...,Belsito,87,16.000000,Sicily & Sardinia,Vittoria,unknown,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,31
7,France,dry restrained wine offers profusion acidity f...,unknown,87,24.000000,Alsace,Alsace,unknown,Roger Voss,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,21
8,Germany,savory thyme notes accent sunnier flavors pres...,Shine,87,12.000000,Rheinhessen,unknown,unknown,Anna Lee C. Iijima,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel,24
9,France,great depth flavor fresh apple pear fruits tou...,Les Natures,87,27.000000,Alsace,Alsace,unknown,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,30


# pre processing

In [2]:
#remove punctuation
data['description'] = data['description'].str.replace('[^\w\s]','')

In [3]:
#remove most frequent words, not influent bcs very frequent
freq = pd.Series(' '.join(data['description']).split()).value_counts()[:10]
freq

wine       72465
flavors    56228
fruit      40686
palate     33691
aromas     32164
acidity    31672
finish     29689
tannins    26477
cherry     23568
ripe       23310
dtype: int64

In [4]:
freq = list(freq.index)
data['description'] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [5]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

data["stopwords"] = data["description"].apply(lambda x: len([x for x in x.split() if x in stop]))

In [6]:
unfreq = pd.Series(' '.join(data["description"]).split()).value_counts()
count = 0
for el in unfreq: 
    if(el < 10):
        count = count + 1
print(count)   #elementi poco frequenti sotto il 10 sono spesso typos/parole unite o robe brutte

30993


In [7]:
unfreqList = unfreq[-count:]
#unfreqList

In [8]:
unfreq = dict(unfreqList)  #con dictionary is veeeeeeeeeeeery fasteroni
data["description"] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in unfreq))

In [9]:
data["word_count"] = data["description"].apply(lambda x: len(str(x).split(" ")))
data.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,word_count,stopwords
0,Italy,tropical broom brimstone herb expressive apple...,Vulkà Bianco,87,17.0,Sicily & Sardinia,Etna,unknown,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,10,0
1,Portugal,fruity firm juicy red berry fruits drinkable,Avidagos,87,15.0,Douro,unknown,unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,7,0
2,US,tart snappy lime flesh dominate green pineappl...,unknown,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,9,0
3,US,pineapple rind lemon pith orange blossom opule...,Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,unknown,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,13,0
4,US,much regular bottling rough tannic rustic eart...,Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,16,0


In [10]:
len(data)

117652

# TF IDF  fulltext da ignorare

fullText = pd.Series(' '.join(data['description']).split()).value_counts()
#fullText

tfData = {}
for idx, row in data.iterrows(): 
    tfData[idx] = pd.Series(row['description'].split()).value_counts();

tf1 = {}
for idx in tfData.keys():
    tf1[idx] = {}
    for word in tfData[idx].keys():
        tf1[idx][word] = tfData[idx][word] / fullText[word]


#idf
docCount = dict.fromkeys(fullText.keys(),0)
idfDict1 = {}
N = len(data)                                #numero di documenti #dizionario contenente tutte le parole presenti 
idfData = dict(tfData)
for el in idfData:     #per ogni descrizione
    for word,val in idfData[el].items():
        docCount[word] = docCount[word] + 1
        
for word in fullText.keys():
    idfDict1[word] = math.log10(N/docCount[word])  

#tf-idf
tfIdf1 = {}
for idx in tfData.keys():
    tfIdf1[idx] = {}
    for word in tfData[idx].keys():
        val = tf1[idx][word] * idfDict1[word]
        tfIdf1[idx][word] = val
tfIdf1[0]

!!!! tf idf devono essere usati sulle varietà, così si capisce per quali qualità sono i vini più importanti. NOn si calcola il "vettore" delle parole nella descrizione mas i cercano le parole nella descrizione e si vedono che peso hanno in quella generale.

calcolari in generale non ha senso, la frequenza delle parole all'interno di uno short text cime questo è sempre 1. 

# tf idf contando l'importanza di certi termini all'interno di una classe di vini
(raw dataset)

fullText = pd.Series(' '.join(data['description']).split()).value_counts()


totalWords = pd.Series("".join(data["description"]).split()).value_counts().sum()
print(totalWords)
t1 = data.groupby(["variety"])
t2 = pd.DataFrame({'text' : t1['description'].apply(lambda x: (x + " ").sum())}).reset_index()
t2["word_count"] = t2["text"].apply(lambda x: len(str(x).split(" ")))
t2 = t2.sort_values('word_count',ascending = False)
#t2.iloc[0]["text"]
#t2

varietyDict = {}
for idx, row in t2.iterrows():
    varietyDict[row["variety"]] = pd.Series(row['text'].split()).value_counts()
#varietyDict["Pinot Gris"]

tfDict = {}
for var in varietyDict:
    temp = {}
    for word in varietyDict[var].keys():
        temp[word] = varietyDict[var][word] / fullText[word]   #compute tf= numero di occorrenze della parola nel documento /numero di occorrenze della stessa parola in tutti i documenti 
    tfDict[var] = temp
#tfDict["Pinot Gris"] 

varList = data["variety"].unique().tolist()

def computeIdf1(dataset):
    a = []
    idf = {}
    for idx,row in dataset.iterrows():
        el = row["variety"]
        docList = data[data["variety"] == el]
        N = len(docList)
        idfDict = {}
        temp = t2[t2["variety"] == el]
        idfDict = dict.fromkeys(temp.iloc[0]["text"].split(),0)
        
        for idx,row in docList.iterrows():
            for word in row["description"].split():
                idfDict[word] += 1
        
        for word, val in idfDict.items():
            idfDict[word] = math.log10(N/float(val))
        a.append([el,N,idfDict])    
        idf[el] = idfDict
    return idf, a

idf,a = computeIdf1(t2)

a[-40] 

C'è un problema coi vini che hanno solo una descrizione, li si leva? idf = 0.

tfIdf = {}
for var in varietyDict:
    temp = {}
    for word in tfDict[var].keys():
        temp[word] = tfDict[var][word] * idf[var][word]
    tfIdf[var] = temp 

dictlist = []
for word,value in tfIdf["Chardonnay"].items():
    temp = [word,value]
    dictlist.append(temp)
dictlist.sort(key=lambda x: x[1],reverse=True)
dictlist
#puligny,chablisienne è un produttore porcodio, bisogna togliere i nomi dei produttori dalle recensioni e il nome della varietà di vino su cui si sta lavorando?

# tf idf con stemming 

In [11]:
data = pd.read_csv("stemmed.csv")
#remove punctuation
data['description'] = data['description'].str.replace('[^\w\s]','')


In [12]:
data["word_count"] = data["description"].apply(lambda x: len(str(x).split(" ")))
data = data[data["word_count"] >= 3] 
data.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,word_count
0,Italy,aroma tropic fruit broom brimston herb palat e...,Vulkà Bianco,87,17.0,Sicily & Sardinia,Etna,unknown,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,16
1,Portugal,ripe fruiti wine firm tannin juici red berri f...,Avidagos,87,15.0,Douro,unknown,unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,12
2,US,tart snappi flavor lime flesh domin green pine...,unknown,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,15
3,US,pineappl rind lemon pith orang blossom aroma p...,Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,unknown,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,18
4,US,much regular bottl rough tannic rustic earthi ...,Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,18


In [13]:
#remove most frequent words, not influent bcs very frequent
freq = pd.Series(' '.join(data['description']).split()).value_counts()[:10]
freq
#numeri sono più alti

wine      74984
flavor    62401
fruit     52898
acid      35519
palat     33793
aroma     33436
finish    31497
tannin    28259
cherri    27524
ripe      24822
dtype: int64

In [14]:
freq = list(freq.index)
data['description'] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [15]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

data["stopwords"] = data["description"].apply(lambda x: len([x for x in x.split() if x in stop]))

In [16]:
unfreq = pd.Series(' '.join(data["description"]).split()).value_counts()
count = 0
for el in unfreq: 
    if(el < 10):
        count = count + 1
print(count)   #elementi poco frequenti sotto il 10 sono spesso typos/parole unite o robe brutte

26449


In [17]:
unfreqList = unfreq[-count:]
#unfreqList

In [18]:
unfreq = dict(unfreqList)  #con dictionary is veeeeeeeeeeeery fasteroni
data["description"] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in unfreq))

In [19]:
fullText = pd.Series(' '.join(data['description']).split()).value_counts()
fullText

black                  22598
note                   22353
rich                   19885
spice                  19749
fresh                  19087
dri                    17090
red                    16397
berri                  15821
oak                    15116
nose                   14395
plum                   13697
sweet                  13315
blackberri             13182
appl                   12914
soft                   12706
textur                 12602
drink                  11787
crisp                  11740
blend                  11667
white                  10922
fruiti                 10804
light                  10388
dark                   10214
citru                   9952
hint                    9823
cabernet                9736
herb                    9607
vanilla                 9536
bright                  9402
raspberri               9228
                       ...  
barriqueag                10
tempranillocabernet       10
actress                   10
dart          

In [20]:
totalWords = pd.Series("".join(data["description"]).split()).value_counts().sum()
print(totalWords)
t1 = data.groupby(["variety"])
t2 = pd.DataFrame({'text' : t1['description'].apply(lambda x: (x + " ").sum())}).reset_index()
t2["word_count"] = t2["text"].apply(lambda x: len(str(x).split(" ")))
t2 = t2.sort_values('word_count',ascending = False)
#t2.iloc[0]["text"]
#t2

1752815


In [21]:
varietyDict = {}
for idx, row in t2.iterrows():
    varietyDict[row["variety"]] = pd.Series(row['text'].split()).value_counts()
#varietyDict["Pinot Gris"]

In [22]:
tfDict = {}
for var in varietyDict:
    temp = {}
    for word in varietyDict[var].keys():
        temp[word] = varietyDict[var][word] / fullText[word]   #compute tf= numero di occorrenze della parola nel documento /numero di occorrenze della stessa parola in tutti i documenti 
    tfDict[var] = temp
#tfDict["Pinot Gris"] 

In [23]:
varList = data["variety"].unique().tolist()

def computeIdf1(dataset):
    a = []
    idf = {}
    for idx,row in dataset.iterrows():
        el = row["variety"]
        docList = data[data["variety"] == el]
        N = len(docList)
        idfDict = {}
        temp = t2[t2["variety"] == el]
        idfDict = dict.fromkeys(temp.iloc[0]["text"].split(),0)
        
        for idx,row in docList.iterrows():
            for word in row["description"].split():
                idfDict[word] += 1
        
        for word, val in idfDict.items():
            idfDict[word] = math.log10(N/float(val))
        a.append([el,N,idfDict])    
        idf[el] = idfDict
    return idf, a

In [24]:
idf,a = computeIdf1(t2)

In [25]:
a[-40] 

['Uvalino',
 1,
 {'assert': 0.0,
  'black': 0.0,
  'clove': 0.0,
  'cocoa': 0.0,
  'deliv': 0.0,
  'dens': 0.0,
  'dust': 0.0,
  'enjoy': 0.0,
  'grape': 0.0,
  'nativ': 0.0,
  'plum': 0.0,
  'rare': 0.0,
  'robust': 0.0,
  'vanilla': 0.0}]

C'è un problema coi vini che hanno solo una descrizione, li si leva? idf = 0.

In [26]:
tfIdf = {}
for var in varietyDict:
    temp = {}
    for word in tfDict[var].keys():
        temp[word] = tfDict[var][word] * idf[var][word]
    tfIdf[var] = temp 

In [27]:
dictlist = []
for word,value in tfIdf["Chardonnay"].items():
    temp = [word,value]
    dictlist.append(temp)
dictlist.sort(key=lambda x: x[1],reverse=True)
dictlist
#con lo stemming cambia un po' le cose

[['chablisian', 3.030032704936171],
 ['puligni', 3.030032704936171],
 ['chablisienn', 3.030032704936171],
 ['fourchaum', 2.9160893526293346],
 ['chablislik', 2.8259127222802465],
 ['chablisstyl', 2.7995837835578974],
 ['serein', 2.751279103983342],
 ['pulignymontrachet', 2.7395866847964507],
 ['kumeu', 2.723862885127889],
 ['cortoncharlemagn', 2.723862885127889],
 ['fuissé', 2.707813410202252],
 ['clarifi', 2.5288492475044158],
 ['cob', 2.525027254113476],
 ['chassagnemontrachet', 2.3307943884124396],
 ['meursault', 2.2930398924095043],
 ['pansear', 2.1530763598325686],
 ['movi', 2.1293932122670474],
 ['challah', 2.084628478629485],
 ['brie', 2.055209252567452],
 ['leflaiv', 2.0385296042003502],
 ['mâcon', 2.0214834883497703],
 ['butteredtoast', 2.0200218032907804],
 ['durel', 1.9672343059256976],
 ['butterscotchi', 1.933825895150436],
 ['brulé', 1.9242724418118327],
 ['brewer', 1.9242724418118327],
 ['went', 1.9194829596136305],
 ['marcona', 1.9092668334379292],
 ['daffodil', 1.895458

In [28]:
weightList = []
c = data.iloc[0]["description"].split()
for word in c:
    weightList.append(tfIdf[data.iloc[0]["variety"]][word])
weightList 

[0.04885833490039738,
 0.3057715203598046,
 0.38444489459092884,
 0.015781246626372144,
 0.018044051307306755,
 0.024294071201374504,
 0.029923012882891484,
 0.012616619865384635,
 0.01593123444525116,
 0.03454049022900759]

In [29]:
variety = data.iloc[0]["variety"]
print(variety)
tfVals = []
for word in c: 
    tfVals.append(tfIdf[variety][word])
    print(word,tfIdf[variety][word])

tfVals.sort(reverse = True)

top3 = tfVals[:3]
top3

White Blend
tropic 0.04885833490039738
broom 0.3057715203598046
brimston 0.38444489459092884
herb 0.015781246626372144
express 0.018044051307306755
appl 0.024294071201374504
citru 0.029923012882891484
dri 0.012616619865384635
sage 0.01593123444525116
brisk 0.03454049022900759


[0.38444489459092884, 0.3057715203598046, 0.04885833490039738]

In [30]:
data["word_count"] = data["description"].apply(lambda x: len(str(x).split(" ")))
data = data[data["word_count"] >= 3] 
print(len(data[data["word_count"] == 3]))
print(len(data))

74
117628


In [31]:
top1 = []
top2 = []
top3 = []
for idx, row in data.iterrows():
    weightList = []
    c = row["description"].split()
    for word in c:
        weightList.append(tfIdf[row["variety"]][word])
    weightList.sort(reverse = True)
    top1.append(weightList[0])
    top2.append(weightList[1])
    top3.append(weightList[2])

In [32]:
data.iloc[9289]

country                                                    Italy
description    distinct prosecco vintag chop white pepper min...
designation                                  Uvaggio Storico Dry
points                                                        87
price                                                         24
province                                                  Veneto
region_1                               Prosecco di Valdobbiadene
region_2                                                 unknown
taster_name                                              unknown
title          Val d'Oca 2007 Uvaggio Storico Dry  (Prosecco ...
variety                                                 Prosecco
winery                                                 Val d'Oca
word_count                                                    17
stopwords                                                      1
Name: 9290, dtype: object

In [33]:
print(len(top1),len(top2),len(top3))

data["top1_desc"] = top1
data["top2_desc"] = top2
data["top3_desc"] = top3


117628 117628 117628


C:\Users\gneur\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\gneur\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\gneur\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [34]:
data.drop(["region_2","word_count","stopwords"],inplace=True,axis=1)

C:\Users\gneur\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [35]:
#data.to_csv("dataWithTop3Desc.csv", encoding='utf-8', index=False)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,top1_desc,top2_desc,top3_desc
0,Italy,tropic broom brimston herb express appl citru ...,Vulkà Bianco,87,17.000000,Sicily & Sardinia,Etna,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,0.384445,0.305772,0.048858
1,Portugal,fruiti firm juici red berri drinkabl,Avidagos,87,15.000000,Douro,unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,0.076411,0.039406,0.038838
2,US,tart snappi lime flesh domin green pineappl po...,unknown,87,14.000000,Oregon,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,0.207796,0.143191,0.095575
3,US,pineappl rind lemon pith orang blossom opul no...,Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,1.425857,0.312786,0.235059
4,US,much regular bottl rough tannic rustic earthi ...,Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,0.741792,0.386043,0.375841
5,Spain,blackberri typic navarran whiff green herb cas...,Ars In Vitro,87,15.000000,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,0.040243,0.028170,0.010061
6,Italy,bright inform candi berri white pepper savori ...,Belsito,87,16.000000,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,0.007392,0.001233,0.001107
7,France,dri restrain offer profus firm textur much food,unknown,87,24.000000,Alsace,Alsace,Roger Voss,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,0.156063,0.068474,0.028350
8,Germany,savori thyme note accent preserv peach brisk o...,Shine,87,12.000000,Rheinhessen,unknown,Anna Lee C. Iijima,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel,0.532834,0.095403,0.022277
9,France,great depth fresh appl pear touch spice dri cr...,Les Natures,87,27.000000,Alsace,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,0.026868,0.021426,0.015970
